In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
# Step 1: Set up file paths and the list of columns to keep
directory = os.path.join(os.getcwd(), 'complete dataset')

# Columns to keep in the final merged file
columns_to_keep = [
    "Date", "ReportYear", "ReportMonth", "OIPID", "SupplierID", 
    "NRENID", "Country", "Currency", "InstitutionName", "ProviderREF", 
    "ServiceType", "EuroCost", "Local Currency", "ExchangeRate", 
    "Local Cost", "ConsumptionType"
]

# List to hold DataFrames from all files
dfs = []

# Track processed files by name (without extension) to avoid duplicates
processed_files = []

In [ ]:
# Step 2: Function to clean and format the data
def clean_and_format_data(df, filename):
    print(f"Processing file: {filename}")
    
    # Keep only the required columns, add missing columns as NaN
    for col in columns_to_keep:
        if col not in df.columns:
            df[col] = np.nan
    
    # Format 'Date' column as 'dd/mm/yyyy'
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.strftime('%d/%m/%Y')
    
    # Extract ReportMonth and ReportYear from Date if missing
    if 'ReportMonth' in df.columns and df['ReportMonth'].isnull().all():
        # Extract month from 'Date' column if 'ReportMonth' is missing
        df['ReportMonth'] = pd.to_datetime(df['Date'], errors='coerce').dt.month
    if 'ReportYear' in df.columns and df['ReportYear'].isnull().all():
        df['ReportYear'] = pd.to_datetime(df['Date'], errors='coerce').dt.year

    # Format 'ReportMonth' to single digit (e.g., '1' instead of '01')
    df['ReportMonth'] = df['ReportMonth'].apply(lambda x: str(int(x)) if pd.notnull(x) else x)
    
    # Format 'ReportYear' to keep only the year, add month to 'ReportMonth' if necessary
    if 'ReportYear' in df.columns:
        df['ReportYear'] = df['ReportYear'].astype(str)
        if 'ReportMonth' in df.columns:
            df['ReportMonth'] = df['ReportYear'].str.extract(r'-(\d+)').fillna(df['ReportMonth'])
        df['ReportYear'] = df['ReportYear'].str.extract(r'(\d{4})')
    
    # Convert 'Country' column to uppercase
    if 'Country' in df.columns:
        df['Country'] = df['Country'].str.upper()
    
    # Ensure 'ExchangeRate' contains only numeric values, default to 1 if non-numeric
    if 'ExchangeRate' in df.columns:
        df['ExchangeRate'] = pd.to_numeric(df['ExchangeRate'], errors='coerce').fillna(1)
    
    # If 'ConsumptionType' is missing, fill with 'ServiceType' values
    if 'ConsumptionType' in df.columns and df['ConsumptionType'].isnull().all():
        df['ConsumptionType'] = df['ServiceType']
    
    # Replace null values in cost columns with 0.0
    df['EuroCost'] = df['EuroCost'].fillna(0.0)
    df['Local Cost'] = df['Local Cost'].fillna(0.0)
    
    # Return only the requested columns
    return df[columns_to_keep]

In [ ]:
# Step 3: Function to process CSV files with multiple encodings
def process_csv(file_path, filename):
    encodings = ['utf-8', 'ISO-8859-1', 'latin1']
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, encoding=encoding)
            df = clean_and_format_data(df, filename)
            return df
        except Exception as e:
            print(f"Error reading CSV file {filename} with {encoding}: {e}")
    return None

In [ ]:
# Step 4: Iterate through the files, preferring CSV over Excel when duplicates exist
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    file_name_without_ext = os.path.splitext(filename)[0]

    # Skip Excel files if we already have the corresponding CSV file
    if file_name_without_ext in processed_files and filename.endswith(".xlsx"):
        continue

    # Process CSV files first and prioritize them
    if filename.endswith(".csv"):
        try:
            df = process_csv(file_path, filename)
            if df is not None and not df.empty:
                dfs.append(df)
                processed_files.append(file_name_without_ext)  # Mark the CSV as processed
                print(f"Successfully processed CSV file: {filename}")
            else:
                print(f"{filename} is empty or could not be read properly.")
        except Exception as e:
            print(f"Error reading CSV file {filename}: {e}")

    # Process Excel files if no corresponding CSV file exists
    elif filename.endswith(".xlsx") or filename.endswith(".xls"):
        if file_name_without_ext not in processed_files:  # Only process Excel if no CSV
            try:
                df = pd.read_excel(file_path)
                if not df.empty:
                    df = clean_and_format_data(df, filename)
                    dfs.append(df)
                    processed_files.append(file_name_without_ext)
                    print(f"Successfully processed Excel file: {filename}")
                else:
                    print(f"{filename} is empty or could not be read properly.")
            except Exception as e:
                print(f"Error reading Excel file {filename}: {e}")

In [ ]:
# Step 5: Merge all DataFrames, remove rows with null 'Date', and save to a CSV file
if dfs:
    merged_df = pd.concat(dfs, ignore_index=True)
    print(f"Final merged data shape before removing null dates: {merged_df.shape}")
    
    # Remove rows with null 'Date' in the final step
    merged_df = merged_df.dropna(subset=['Date'])
    
    print(f"Final merged data shape after removing null dates: {merged_df.shape}")
    
    # Save the merged DataFrame to a CSV file
    merged_output_path = os.path.join(directory, 'merged_output.csv')
    merged_df.to_csv(merged_output_path, index=False)
    print(f"Merged file saved as {merged_output_path}")
else:
    print("No valid files found or loaded.")